In [25]:
import pandas as pd
import matplotlib.pyplot as plt


In [26]:
data = {}

for year in range(2010,2019):
    print(year)
    d = pd.read_csv('./data/final_dataset/merged_'+str(year)+'.csv', dtype={'GeoID': str, 'Tract':str, 'Total Population with Bachelor Degree or Higher':float})
    if year == 2010:
        d.rename(columns={d.columns[15] : 'Total_Population'}, inplace = True)

    for name in d.columns:
        d.rename(columns={name : name.replace(' ', '_')}, inplace = True)
    d.set_index('GeoID', inplace=True)
    data[str(year)] = d
gen_tracts = pd.read_csv('./data/gentrified_tract/new_gentrified_tracts.csv', dtype = {'geoid' : str, 'tract' : str})
gen_tracts = gen_tracts.rename(columns = {'geoid' : 'GeoID', 'tract' : 'TRACT'})


2010
2011
2012
2013
2014
2015
2016
2017
2018


In [54]:
change_data={}
for year in range(2011,2019):
    print(str(year))
    df = pd.DataFrame()
    df['GeoID'] = data[str(year)].index
    df['Tract'] = data[str(year)]['Tract'].values
    df['Change_household_income'] = data[str(year)]['Increase_in_household_income'].values
    df['Change_home_value'] = data[str(year)]['Increase_in_home_value'].values
    
    for feature in [ 'Median_household_income','Median_home_value','calls_DOB', 'calls_NYPD', 'calls_DPR', 'calls_DOT','calls_DSNY', 'calls_TLC', 'calls_DOHMH', 'calls_DCA','calls_DEP', 'calls_DOE', 'calls_EDC', 'calls_DOITT', 'calls_DFTA','Total_Population','Total_Population_with_Bachelor_Degree_or_Higher', 'Number_of_non-Hispanic_Caucasians', 'Number_of_African_Americans','Number_of_American_Indians_and_Alaskans', 'No_of_non-Hispanic_Asians','Number_of_non-Hispanic_Hawaiians', 'No_of_non-Hispanic_others','No_of_non-Hispanic_multi-racials', 'No_of_Hispanics_or_Latinos']:
        col1=data[str(year)][feature].values
        col2=data[str(year-1)][feature].values
        df['Change_'+feature] = col1/col2
    change_data[str(year)] = df

2011
2012
2013
2014
2015
2016
2017
2018


<ipython-input-54-3c3c15af2357>:13: RuntimeWarning: divide by zero encountered in true_divide
  df['Change_'+feature] = col1/col2
<ipython-input-54-3c3c15af2357>:13: RuntimeWarning: invalid value encountered in true_divide
  df['Change_'+feature] = col1/col2


In [108]:
from tqdm._tqdm_notebook import tqdm_notebook
import numpy as np
import math
ml_data={}

def avg_percent_change(row,feature, change_data, year):
    changes = []
    geoid=row.GeoID
    while year!= 2010:
        dat=change_data[str(year)]
        change = dat[dat['GeoID']==geoid][feature].values[0]
        if (~math.isinf(change)) & (pd.notnull(change)):
            changes.append(change)
        year = year - 1
    try:
        return (np.array(changes).prod()**(1/len(changes)))*100
    except ZeroDivisionError:
        return np.nan

# Tract wasn't being gentrified if the percentage increase in income and house values didn't meet the threshold for 2 years in a row
def calc_years(row, change_data, year, income_increase_threshold = 1.1, houseprice_increase_threshold = 1.1):
    years = 1
    geoid=row.GeoID
    curr_change = change_data[str(year)]
    curr_increase_income = curr_change[curr_change['GeoID'] == geoid ]['Change_Median_household_income'].values[0]
    curr_increase_houseprice = curr_change[curr_change['GeoID'] == geoid ]['Change_Median_home_value'].values[0]
    while ((year != 2010) & (curr_increase_income >= income_increase_threshold) & (curr_increase_houseprice >= houseprice_increase_threshold)):
        curr_change = change_data[str(year)]
        curr_increase_income = curr_change[curr_change['GeoID'] == geoid ]['Change_Median_household_income'].values[0]
        curr_increase_houseprice = curr_change[curr_change['GeoID'] == geoid ]['Change_Median_home_value'].values[0]
        years = years+1
        year = year-1
    if year != 2010:
        curr_change = change_data[str(year)]
        curr_increase_income = curr_change[curr_change['GeoID'] == geoid ]['Change_Median_household_income'].values[0]
        curr_increase_houseprice = curr_change[curr_change['GeoID'] == geoid ]['Change_Median_home_value'].values[0]
        while ((year != 2010) & (curr_increase_income >= income_increase_threshold) & (curr_increase_houseprice >= houseprice_increase_threshold)):
            curr_change = change_data[str(year)]
            curr_increase_income = curr_change[curr_change['GeoID'] == geoid ]['Change_Median_household_income'].values[0]
            curr_increase_houseprice = curr_change[curr_change['GeoID'] == geoid ]['Change_Median_home_value'].values[0]
            years = years+1
            year = year-1
    return years
    
    
for year in range(2011,2019):
    print(str(year))
    df = pd.DataFrame()
    df['GeoID'] = gen_tracts[gen_tracts['Year'] == year]['GeoID'].values
    df['Tract'] = gen_tracts[gen_tracts['Year'] == year]['TRACT'].values
    gen_values = {}
  
    for feature in ['Change_Median_household_income', 'Change_Median_home_value','Change_calls_DOB', 'Change_calls_NYPD', 'Change_calls_DPR','Change_calls_DOT', 'Change_calls_DSNY', 'Change_calls_TLC','Change_calls_DOHMH', 'Change_calls_DCA', 'Change_calls_DEP','Change_calls_DOE', 'Change_calls_EDC', 'Change_calls_DOITT','Change_calls_DFTA', 'Change_Total_Population','Change_Total_Population_with_Bachelor_Degree_or_Higher','Change_Number_of_non-Hispanic_Caucasians','Change_Number_of_African_Americans','Change_Number_of_American_Indians_and_Alaskans','Change_No_of_non-Hispanic_Asians','Change_Number_of_non-Hispanic_Hawaiians','Change_No_of_non-Hispanic_others','Change_No_of_non-Hispanic_multi-racials','Change_No_of_Hispanics_or_Latinos']:
        df['avg_'+feature] = df.apply(lambda row: avg_percent_change(row, feature, change_data, year), axis = 1)
    df['yrs_to_gentrify'] = df.apply(lambda row: calc_years(row, change_data, year), axis = 1)
    ml_data[str(year)] = df
        
    
    
    
    
    
    
    
    

2011
2012


<ipython-input-108-ee1a6b415845>:16: RuntimeWarning: invalid value encountered in double_scalars
  return (np.array(changes).prod()**(1/len(changes)))*100


2013
2014
2015
2016
2017
2018


In [111]:
ml_data['2015']

,GeoID,Tract,avg_Change_Median_household_income,avg_Change_Median_home_value,avg_Change_calls_DOB,avg_Change_calls_NYPD,avg_Change_calls_DPR,avg_Change_calls_DOT,avg_Change_calls_DSNY,avg_Change_calls_TLC,...,avg_Change_Total_Population_with_Bachelor_Degree_or_Higher,avg_Change_Number_of_non-Hispanic_Caucasians,avg_Change_Number_of_African_Americans,avg_Change_Number_of_American_Indians_and_Alaskans,avg_Change_No_of_non-Hispanic_Asians,avg_Change_Number_of_non-Hispanic_Hawaiians,avg_Change_No_of_non-Hispanic_others,avg_Change_No_of_non-Hispanic_multi-racials,avg_Change_No_of_Hispanics_or_Latinos,yrs_to_gentrify
0,36005004800,4800,70.274045,70.133116,0.000000,257.541520,0.000000,142.218947,267.289077,0.000000,...,47.490769,20.126135,192.160110,NaN,48.010185,NaN,0.000000,0.000000,857.315787,1
1,36005021700,21700,47.233894,NaN,252.225001,259.765725,109.891525,56.747740,61.337485,0.000000,...,57.694304,9.896560,553.473219,0.000000,0.000000,NaN,19.512195,109.981055,533.241977,1
2,36047112200,112200,48.425572,NaN,234.034732,227.970986,38.614439,43.668480,0.000000,0.000000,...,52.453137,62.923799,464.694928,0.000000,51.196398,NaN,0.000000,0.000000,232.209803,1
3,34013000200,200,67.553632,84.993246,NaN,NaN,NaN,NaN,NaN,NaN,...,56.400039,52.078965,59.466674,0.000000,75.868324,0.0,155.528741,174.934762,147.926444,1
4,36047048200,48200,56.788210,NaN,99.186884,246.036842,58.245263,210.302145,243.849948,141.421356,...,115.507818,450.087752,42.422990,0.000000,253.011574,NaN,309.375000,93.336912,47.444509,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,36005039600,39600,83.352936,107.189716,135.096004,144.954435,32.111478,53.128265,370.779275,0.000000,...,50.649664,15.900282,141.600301,0.000000,0.000000,NaN,159.248330,0.000000,61.911647,1
219,36061030900,30900,46.858797,77.601075,131.037070,226.859983,138.253401,35.574973,0.000000,0.000000,...,121.266719,40.896788,1055.689489,67.480156,8.842364,NaN,0.000000,98.158520,1315.202243,1
220,36081097204,97204,80.379419,71.693646,0.000000,51.556470,26.520079,98.212271,0.000000,0.000000,...,43.581405,38.910746,693.837457,0.000000,72.477771,NaN,324.509354,152.917929,59.790691,1
221,36081015801,15801,109.917639,86.916475,174.787027,106.108927,91.624644,65.310520,0.000000,0.000000,...,26.064817,4.771198,235.569903,105.953141,210.350930,NaN,2214.347394,125.614610,41.770311,1


In [113]:
for year in ml_data.keys():
    ml_data[year].to_csv('./data/final_dataset/ml_data/ml_'+year+'.csv', index=False)